In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add, Input, Flatten
from keras import optimizers, initializers, regularizers, metrics
from keras import backend as K

import os
import shutil
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
from turtle import st


input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

def conv_layer_1(x):
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)

    return x

def conv_layer_2(x):
    x = MaxPooling2D((3, 3), 2)(x)
    shortcut = x

    for i in range(3):
        if (i == 0):
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)
            shortcut = BatchNormalization()(shortcut)
        
        x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        shortcut = x
        
    return x

def conv_layer_3(x):
    for i in range (4):
        if (i == 0):
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            shortcut = BatchNormalization()(shortcut)
        
        x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)

        x = Add()([x, shortcut])(x)
        x = Activation('relu')(x)

        shortcut = x

    return x

def conv_layer_4(x):
    for i in range(6):
        if (i == 0):
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            shortcut = BatchNormalization()(shortcut)
        
        x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)

        x = Add()([x, shortcut])(x)
        x = Activation('relu')(x)

        shortcut = x
    
    return x

def conv_layer_5(x):
    for i in range(3):
        if (i == 0):
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            shortcut = BatchNormalization()(shortcut)

        x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')
        x = BatchNormalization()(x)
        
        x = Add()([x, shortcut])(x)
        x = Activation('relu')(x)

        shortcut = x
    
    return x

x = conv_layer_1(input_tensor)
x = conv_layer_2(x)
x = conv_layer_3(x)
x = conv_layer_4(x)
x = conv_layer_5(x)

x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)



